<a href="https://colab.research.google.com/github/lamvng/network-anomaly-dectection/blob/tuan_edit/Spark_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sử dụng KMeans triển khai trên Pyspark để phát hiện bất thường trong mạng
*Thái Bá Tuân, Nguyễn Văn Lâm, Phạm Anh Đức - PFIEV K60*


## **0. Giới thiệu và trình tự bài báo cáo**
Bài báo cáo được trình bày thành ba phần như sau:

**1.   Giới thiệu mục tiêu và bộ dữ liệu**

*   Giới thiệu bài toán Anomalies Detection
*   Bộ dữ liệu `kddcup 1999`

**2.   Thuật toán KMeans**

*   Giới thiệu
*   Thuật toán

**3.   Triển khai KMeans bằng Pyspark**

*   Khảo sát và tiền xử lý bộ dữ liệu
*   Chạy KMeans với 2 clusters và nhận xét
*   Lựa chọn số clusters phù hợp
*   Xác định Anomalies với số cụm đã chọn



## **1. Giới thiệu mục tiêu và bộ dữ liệu**

### ***1.1. Mục tiêu bài toán***

Với dữ liệu đầu vào là một tập các log files ghi lại các kết nối của một hệ thống nhất định, ta cần đưa ra dự đoán xem kết nối đó có phải bất thường hay không (mass ping, quét cổng hàng loạt...)


*   **Input**: Log file ghi lại các kết nối tới hệ thống, bao gồm các thông tin như IP, giao thức kết nối, kích thước gói tin...
*   **Output**: Dự đoán các kết nối đó có phải kết nối bất thường (brute force mật khẩu, quét cổng hàng loạt, mass ping...) hay không.

### ***1.2. Giới thiệu bộ dữ liệu KDD 1999***
Bộ dữ liệu được nhóm sử dụng cho bài báo cáo là [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html), trong đó, bao gồm các file như sau:

*   [kddcup.data.gz](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz): Bộ dữ liệu bản đầy đủ. Kích thước file nén là 18 MB, khi giải nén ra sẽ thành 743 MB.
*   [kddcup.data_10_percent.gz](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz): Bộ dữ liệu có kích thước bằng 10% bản đầy đủ. Kích thước file nén là 2 MB, khi giải nén ra sẽ thành 75 MB.
*   [kddcup.names](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names): Chứa tên và kiểu dữ liệu của mỗi trường có trong bộ dữ liệu. Tệp dữ liệu `kddcup.data.gz` không có sẵn những thông tin về tên trường (*header*) tên ta có thể thêm chúng vào dataset ở bước tiền xử lý.

Bộ dữ liệu `kddcup.data.gz` được lưu dưới định dạng `csv`, bao gồm 4,9 triệu bản ghi, mỗi bản ghi bao 
Để tăng tốc độ load và xử lý dữ liệu, nhóm sẽ triển khai thuật toán trên tập dữ liệu 10%, có kích thước 75 MB, thay vì bộ dữ liệu gốc có kích thước 743 MB.



## **2. Thuật toán KMeans**
 
## ***2.1. Giới thiệu***
Phân cụm là một loại hình học không giám sát (unsupervised learning) điển hình. Mục đích là làm thể nào để phân dữ liệu thành các cụm (cluster) khác nhau sao cho dữ liệu trong cùng một cụm có tính chất giống nhau.

Phân cụm K-Means là một thuật toán phân cụm đơn giản và được sử dụng rộng rãi. Cho giá trị của k, nó cố gắng xây dựng các cụm k từ các mẫu trong bộ dữ liệu. Do đó, k là một siêu tham số của mô hình. Giá trị đúng của k không dễ xác định, vì nó phụ thuộc nhiều vào tập dữ liệu và cách dữ liệu được thực hiện.

Để kiểm tra sự giống nhau giữa hai điểm dữ liệu bất kỳ, K-mean yêu cầu định nghĩa về khoảng cách giữa các điểm dữ liệu. Khoảng cách là gì? Đó là một giá trị cho biết mức độ gần nhau của hai điểm dữ liệu trong không gian của chúng. Đặc biệt, khi các điểm dữ liệu nằm trong một không gian hai chiều, khoảng cách Euclide là một lựa chọn tốt của hàm khoảng cách và được thư viện MLLIB hỗ trợ.

Trong K-mean, cụm là một nhóm các điểm, với một thực thể đại diện được gọi là centroid. Một centroid cũng là một điểm trong không gian dữ liệu: trung tâm của tất cả các điểm tạo nên cụm. Nó được định nghĩa là trung bình số học của các điểm. Nói chung, khi làm việc với K-means-clustering, mỗi mẫu dữ liệu được biểu diễn trong một vectơ số d chiều, do đó dễ dàng hơn để xác định hàm khoảng cách phù hợp. Kết quả là, trong một số ứng dụng, dữ liệu gốc phải được chuyển đổi thành một biểu diễn khác, để phù hợp với yêu cầu của phương tiện K.
### ***2.1.2   Thuật toán K-means***
**Đầu vào:** Dữ liệu X và số lượng cluster cần tìm k.

**Đầu ra:**  Các center M và label vector cho từng điểm dữ liệu Y


1.   Chọn K điểm bất kỳ làm các center ban đầu.
2.   Phân mỗi điểm dữ liệu vào cluster có center gần nó nhất.
3.   Nếu việc gán dữ liệu vào từng cluster ở bước 2 không thay đổi so với vòng lặp trước nó thì ta dừng thuật toán.
4.   Cập nhật center cho từng cluster bằng cách lấy trung bình cộng của tất các các điểm dữ liệu đã được gán vào cluster đó sau bước 2.
5.   Quay lại bước 2.

Chúng ta cũng có thể chấm dứt thuật toán khi nó đạt đến số lần lặp xác định, mang lại kết quả gần đúng. Từ mã giả của thuật toán, chúng ta có thể thấy rằng kết quả phân cụm có nghĩa là **K** có thể nhạy cảm với thứ tự mà các mẫu dữ liệu trong tập dữ liệu được khám phá. Một thực hành hợp lý sẽ là chạy phân tích nhiều lần, sắp xếp ngẫu nhiên các đối tượng; sau đó, tính trung bình các trung tâm cụm của các lần chạy đó và nhập các trung tâm làm trung tâm ban đầu cho một lần phân tích cuối cùng.



## **3. Triển khai KMeans bằng Pyspark**

---


Đầu tiên, cần kích hoạt [Google Colab](https://colab.research.google.com/) bằng tài khoản Google. Các file sau đó sẽ được mở dưới dạng Jupyter Notebook.

Sau đó ta có thể tiến hành cài đặt Pyspark trực tiếp qua Google Colab thông qua thư viện `findspark`.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Cài đặt các biến môi trường:

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



Kết nối tới tài khoản Drive, cho phép sử dụng Drive API:

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Tạo PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



```
# This is formatted as code
```

Lấy link bộ dữ liệu:

In [0]:
link = "https://drive.google.com/open?id=1wOl76pErCKeFwFnPRk_u-56STpqva2JQ" # You may change your URL here
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1wOl76pErCKeFwFnPRk_u-56STpqva2JQ


Tạo SparkSession và lấy bộ dữ liệu:

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('kddcup.csv')

In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Anomalies Detection") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Bộ dữ liệu gốc được lưu dưới dạng file `csv` nhưng không có header. Vì vậy, ta cần tự thêm header vào các trường trong `DataFrame`.

In [0]:
dataWithoutHeader = spark.read.option("inferSchema", True).option("header", False).csv("kddcup.csv")
data = dataWithoutHeader.toDF("duration", "protocol_type", "service", "flag",
"src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
"dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
"label")

Bộ dữ liệu có dạng như sau:

In [0]:
data.show(10)

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_hos

Các nhãn dữ liệu và số lượng tương ứng:

In [0]:
from pyspark.sql.functions import col
data.select("label").groupBy("label").count().orderBy(col("count").desc()).show(25)

+----------------+------+
|           label| count|
+----------------+------+
|          smurf.|280790|
|        neptune.|107201|
|         normal.| 97278|
|           back.|  2203|
|          satan.|  1589|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|    warezclient.|  1020|
|       teardrop.|   979|
|            pod.|   264|
|           nmap.|   231|
|   guess_passwd.|    53|
|buffer_overflow.|    30|
|           land.|    21|
|    warezmaster.|    20|
|           imap.|    12|
|        rootkit.|    10|
|     loadmodule.|     9|
|      ftp_write.|     8|
|       multihop.|     7|
|            phf.|     4|
|           perl.|     3|
|            spy.|     2|
+----------------+------+



`import` các hàm cần thiết từ thư viện `pyspark.ml`:

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.clustering import KMeans
from pyspark.ml.clustering import KMeansModel

Trong bộ dữ liệu có những trường không mang giá trị số thực. Ta sẽ tiến hành loại bỏ những trường này và giữ lại những trường có giá trị thực để đưa vào feature.

*   Input: Bộ dữ liệu dưới dạng `Dataframe`.
*   Output: Bộ dữ liệu cũng dạng `Dataframe` sau khi loại bỏ các trường `protocol_type`, `service`, `flag`.

In [0]:
numericOnly = data.drop("protocol_type", "service", "flag")
print(numericOnly.columns)

['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label']


Gộp các trường dữ liệu trong `Dataframe` thành một vector duy nhất, mỗi vector tương ứng với một dòng trong tập dữ liệu.

Ta sử dụng transformation `VectorAssembler`, giúp gộp nhiều cột thành một vector, mỗi phần tử của vector tương ứng với một dòng của `Dataframe` ban đầu.


In [0]:
assembler = VectorAssembler(inputCols=numericOnly.columns[0:-1], outputCol="featureVector")

Tạo ra một mô hình Kmeans với:

*   `setSeed(10)`: Khởi tạo và cố định vị trí trung tâm ban đầu của các cluster.
*   `setPredictionCol("cluster")`: Tạo ra cột mới `cluster`, tương ứng với cụm phân loại của các phần tử trong `featureVector`.
*   `setFeaturesCol("featureVector")`: Đặt các trường feature của mô hình là `featureVector`.



In [0]:
kmeans = KMeans().setSeed(10).setPredictionCol("cluster").setFeaturesCol("featureVector")

[Pipeline](https://spark.apache.org/docs/latest/ml-pipeline.html): Một chuỗi các Transformers và Estimators trong một model học máy.

*   Các hành động: `assembler` và `kmeans`
*   Được thực hiện trên bộ data  `numericOnly`



In [0]:
pipeline = Pipeline(stages=[assembler, kmeans]) # A chain of commands
pipelineModel = pipeline.fit(numericOnly) # A model after performing KMeans

Model Kmeans sau khi đã train xong:

In [0]:
kmeansModel = pipelineModel.stages[-1] # final stage of pipelineModel
print(type(kmeansModel))

<class 'pyspark.ml.clustering.KMeansModel'>


In toạ độ các centers của các cụm:

Mặc định của Kmeans model chỉ phân 2 cụm.

In [0]:
centers = kmeansModel.clusterCenters() # Output: A numpy array of centers
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[4.79793956e+01 1.62207883e+03 8.68534183e+02 4.45326100e-05
 6.43293794e-03 1.41694668e-05 3.45168212e-02 1.51815716e-04
 1.48247035e-01 1.02121372e-02 1.11331525e-04 3.64357718e-05
 1.13517671e-02 1.08295211e-03 1.09307315e-04 1.00805635e-03
 0.00000000e+00 0.00000000e+00 1.38658354e-03 3.32286248e+02
 2.92907143e+02 1.76685418e-01 1.76607809e-01 5.74330999e-02
 5.77183920e-02 7.91548844e-01 2.09816404e-02 2.89968625e-02
 2.32470732e+02 1.88666046e+02 7.53781203e-01 3.09056111e-02
 6.01935529e-01 6.68351484e-03 1.76753957e-01 1.76441622e-01
 5.81176268e-02 5.74111170e-02]
[2.0000000e+00 6.9337564e+08 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 5.7000000e+01
 3.0000000e+00 7.9000000e-01 6.7000000e-01 2.1000000e-01 3.3000000e-01
 5.0000000e-02 3.9000000e-01 0.0000000e+00 2.5

In [0]:
withCluster = pipelineModel.transform(numericOnly)
withCluster.select("cluster", "label").groupBy("cluster", "label").count().orderBy(col("cluster"), col("count").desc()).show(25)

+-------+----------------+------+
|cluster|           label| count|
+-------+----------------+------+
|      0|          smurf.|280790|
|      0|        neptune.|107201|
|      0|         normal.| 97278|
|      0|           back.|  2203|
|      0|          satan.|  1589|
|      0|        ipsweep.|  1247|
|      0|      portsweep.|  1039|
|      0|    warezclient.|  1020|
|      0|       teardrop.|   979|
|      0|            pod.|   264|
|      0|           nmap.|   231|
|      0|   guess_passwd.|    53|
|      0|buffer_overflow.|    30|
|      0|           land.|    21|
|      0|    warezmaster.|    20|
|      0|           imap.|    12|
|      0|        rootkit.|    10|
|      0|     loadmodule.|     9|
|      0|      ftp_write.|     8|
|      0|       multihop.|     7|
|      0|            phf.|     4|
|      0|           perl.|     3|
|      0|            spy.|     2|
|      1|      portsweep.|     1|
+-------+----------------+------+



##**3. Lựa chọn K phù hợp **

**2 clusters rõ ràng là không đủ ở đây. Trong dữ liệu, chúng ta thấy có 23 nhãn được gán, vì vậy số lượng cụm được chia có thế là 23 hoặc thậm chí là nhiều hơn. Thông thường, nhiều giá trị của K sẽ được thử để lựa chọn ra gía trị tốt nhất. Vậy tìm k như thế nào, bao nhiêu?**

- Một cụm có thể được coi là tốt nếu mỗi điểm dữ liệu ở gần trung tâm của nó nhất so với các tâm còn lại, trong đó, gần nhất ở đây được xác định bởi khoảng cách Euclide. Đây là một cách đơn giản, phổ biến để đánh giá chất lượng của một cụm, bằng giá trị trung bình của các khoảng cách này trên tất cả các điểm, hoặc đôi khi, giá trị trung bình của bình phương khoảng cách . 

- KMeansModel cung cấp một phương thức computeCost để tính tổng bình phương khoảng cách và dễ dàng sử dụng để tính bình phương khoảng cách trung bình. Ngoài ra có nhiều cách để tính toán chất lượng của một cụm kmean. 

In [0]:
from pyspark.sql import DataFrame
from random import randrange
def clusteringScore0(data, k):
    assembler = VectorAssembler(inputCols=data.columns[0:-1], outputCol="featureVector")
    kmeans = KMeans(seed=1, k=k, predictionCol="cluster", featuresCol="featureVector")
    pipeline = Pipeline(stages=[assembler, kmeans])
    kmeansModel = pipeline.fit(data).stages[-1]
    print(k, kmeansModel.computeCost(assembler.transform(data)) / data.count()) 

Chúng ta sẽ thực hiện tính toán trên số lượng cụm là "20, 40, 60, 80, 100"

In [0]:
for k in range(20, 160, 20):
    clusteringScore0(numericOnly, k)
    clusteringScore0(numericOnly, k)

//Mark Nhận xét về kết quả ở đây, sau khi chạy lại 

Đề tăng độ chính xác, chúng ta sẽ thêm một số thông số của model K-means. 
1.   tol: Dung sai hội tụ
Đây là tham số cho phép điều khiển lượng chuyển động trung tâm cụm tối thiểu được coi là đáng kể; các giá trị thấp hơn có nghĩa là thuật toán K-mean sẽ cho phép các trọng tâm di chuyển ít hơn
2.   maxIter: Số vòng lặp tối đa.

Mặc định của thuật toán là `maxIter = 20` và `tol =  1e-4`. Chúng ta sẽ đặt `maxIter = 40` và `tol = 1e-5` trong code 


In [0]:
def clusteringScore1(data, k):
    assembler = VectorAssembler(inputCols=data.columns[0:-1], outputCol="featureVector")
    kmeans = KMeans(seed=1, k=k, predictionCol="cluster", featuresCol="featureVector", maxIter=40, tol=1.0e-05) # set số vòng lặp, sai số chấp nhận để dừng vòng lặp
    pipeline = Pipeline(stages=[assembler, kmeans])
    kmeansModel = pipeline.fit(data).stages[-1]
    print(k, kmeansModel.computeCost(assembler.transform(data)) / data.count()) #  

In [0]:
for k in range(20, 120, 20):
    clusteringScore0(numericOnly, k)
    clusteringScore1(numericOnly, k)

20 70090529.18766987
20 70090529.18766987
40 34134989.30719846
40 34134989.30719846
60 32241636.217287987
60 32241469.66946356
80 31426292.4634663
80 31426292.4634663
100 29985935.77407783
100 26300705.9737499


In [0]:
//mark Đưa ra nhận xét.
Test thử xem git hub đã hoạt động chưa.

### 3.1 Kỹ thuật Feature Normalization
Các điểm dữ liệu đôi khi được đo đạc với những đơn vị khác nhau, m và feet chẳng hạn. Hoặc có hai thành phần (của vector dữ liệu) chênh lệch nhau quá lớn, một thành phần có khoảng giá trị từ 0 đến 1000, thành phần kia chỉ có khoảng giá trị từ 0 đến 1 chẳng hạn. Lúc này, chúng ta cần chuẩn hóa dữ liệu trước khi thực hiện các bước tiếp theo.

normalizedi = f eaturei − μi σi

trong đó u là trung bình và o là phương sai của dữ liệu 

Thư viện MLlib cung cấp thư viện `StandardScaler`, dùng để chuẩn hoá dữ liệu, và dễ dàng được thêm vào pipeline, các tham số:
1. withStd(default: True)  Scales the data to unit standard deviation.
2. withMean(default: False) Centers the data with mean before scaling. It will build a dense output, so take care when applying to sparse input
- Một cách tiếp cận khác để xác định độ tốt của các cụm kmean - sử dụng căn bậc hai của tổng phương sai

In [0]:
from pyspark.ml.feature import StandardScaler
def clusteringScore2(data, k):
  assembler = VectorAssembler(inputCols=data.columns[0:-1], outputCol="featureVector")
  scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatureVector", withStd=True, withMean=False)
  kmeans = KMeans(seed=1, k=k, predictionCol="cluster", featuresCol="scaledFeatureVector", maxIter=40, tol=1.0e-05)
  pipeline = Pipeline(stages=[assembler, scaler, kmeans])
  pipelineModel = pipeline.fit(data)
  kmeansModel = pipelineModel.stages[-1]
  print(k, kmeansModel.computeCost(pipelineModel.transform(data)) / data.count())

Chạy thuật toán với số cụm là "30, 60, ..., 270"

In [0]:
for k in range(60, 300, 30):
  clusteringScore2(numericOnly, k)

60 1.1868969161190808
90 0.716842007702905
120 0.49734934242366263
150 0.3759175320833906
180 0.3172011058253099
210 0.273636909577068
240 0.23059890300407695
270 0.20361107540315934


Nhận xét: 


*   Giảm cost
*   Số cụm càng tăng, chi phí càng giảm => chưa hợp lý(hội tụ)
    
### 3.2 Categorical Variables
Ở phần trên chúng ta đã sử dụng phương pháp `Normalization`, tuy nhiên có nhiều phương pháp nữa có thể áp dụng ở đây. Ở phần đầu của bài toán, chúng ta đã loại bỏ đi 3 tham số vì đó không phải kiểu dạng số, chúng ta sẽ sử dụng một số kỹ thuật để sử dụng 3 tham số này, nâng cao độ chính xác của bài toán. 
Kỹ thuật sử dụng ở đây sẽ là **one-hot encoding**: 
Ví dụ với trường protocol_type: có chứa 3 loại là: tcp, udp, hoặc là icmp.
Chúng ta có thể biến đổi 3 loại này về:
tcp => [1, 0, 0]
udp => [0, 1, 0]
icmp => [0, 0, 1]

Cách triển khai:
MLLib cung cấp đủ các thư viện để thực hiện one-hot:
     - Đầu tiên chúng ta sử dụng `StringIndexer` để biến các chuỗi `string` thành số 0, 1, 2, 3, .. tương ứng
     - Tiếp theo sử dụng `OneHotEncoder` để biến các số này thành vector.
Hai giai đoạn này sẽ được sử cho vào 1 `Pipeline`

In [0]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
import math
def oneHotPipeline(inputCol):
    indexer = StringIndexer(inputCol=inputCol, outputCol=(inputCol + "_indexed"))
    encoder = OneHotEncoder(inputCol=inputCol + "_indexed", outputCol=inputCol + "_vec")
    pipeline = Pipeline(stages=[indexer, encoder])
    return (pipeline, inputCol + "_vec")

//Mark: thêm vào demo ở đây để nhìn rõ.

In [0]:
def clusteringScore3(data, k):
    (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline("protocol_type")
    (serviceEncoder, serviceVecCol) = oneHotPipeline("service")
    (flagEncoder, flagVecCol) = oneHotPipeline("flag")
    assembleCols = list(set(data.columns) - set(["label", "protocol_type", "service", "flag"])) + list([protoTypeVecCol, serviceVecCol, flagVecCol])
    assembler = VectorAssembler(inputCols=assembleCols, outputCol="featureVector")
    scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatureVector", withStd=True, withMean=False)
    kmeans = KMeans(seed=1, k=k, predictionCol="cluster", featuresCol="scaledFeatureVector", maxIter=40, tol=1.0e-05)
    pipeline = Pipeline(stages=[protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scaler, kmeans])
    pipelineModel = pipeline.fit(data)
    kmeansModel = pipelineModel.stages[-1]
    print(k, kmeansModel.computeCost(pipelineModel.transform(data)) / data.count())

for k in range(60, 270, 30):
    clusteringScore3(data, k)
    numericOnly.unpersist()

60 34.32659398810193
90 10.102528717704127
120 2.9569851351736243
150 2.1403963857555337
180 1.555682536994971
210 1.2852764808975832
240 0.9366045728591523


In [0]:
import math
def entropy(counts):
    values = filter(lambda x: x > 0, counts)
    n = float(sum(values))
    sum_entropy = 0.0
    for v in counts:
      sum_entropy += -(v/n) * math.log((v/n))
    # e = map(lambda v: -(v/n) * math.log((v/n)), values)
    # print(sum(e))
    return sum_entropy

In [0]:
from itertools import groupby
def list_group(group_list):
  groups = []
  for key, group in groupby(group_list):
    groups.append(len(list(group)))
  return groups


In [0]:
def fitPipeline4(data, k):
  (protoTypeEncoder, protoTypeVecCol) = oneHotPipeline("protocol_type")
  (serviceEncoder, serviceVecCol) = oneHotPipeline("service")
  (flagEncoder, flagVecCol) = oneHotPipeline("flag")
  assembleCols = list(set(data.columns) - set(["label", "protocol_type", "service", "flag"])) + list([protoTypeVecCol, serviceVecCol, flagVecCol])
  assembler = VectorAssembler(inputCols=assembleCols, outputCol="featureVector")
  scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatureVector", withStd=True, withMean=False)
  kmeans = KMeans(seed=1, k=k, predictionCol="cluster", featuresCol="scaledFeatureVector", maxIter=40, tol=1.0e-05)
  pipeline = Pipeline(stages=[protoTypeEncoder, serviceEncoder, flagEncoder, assembler, scaler, kmeans])
  return pipeline.fit(data)

**GroupByKey, MapGroups chỉ hỗ trợ trong Spark với Scala, Java. Tuy nhiên trong document của spark có hướng dẫn là python có những hàm mạnh mẽ tương đương, cụ thể ở đây chúng ta sẽ dùng mapGroup, map, sum của itertools**  
Hàm dưới đây có chức năng tính entropy của 1 tập dữ liệu, công thức tính sẽ là: 
Mỗi cluster được phân cụm sẽ gồm nhiều label trùng nhau, tách tưng cluster ra riêng


1.   Tính size của từng cluster, 
2.   Group từng cluster lại thành các labels, với số lần xuất hiện trong cluster
3.   `entropy[i] = -(frequence/sum)*log((frequence/sum))`
4.   `Score_of_cluster[i] = size[i] * entropy[i] `
5.   `Sum(Score_of_cluster[i])`



In [0]:
def clusteringScore4(data, k):
  pipelineModel = fitPipeline4(data, k)
  # Predict cluster for each datum
  clusterLabel = pipelineModel.transform(data).select("cluster", "label")
  labelsInCluster = clusterLabel.rdd.groupByKey().values()
  labelCounts = labelsInCluster.map(lambda labels: list_group(labels)).collect()
  sum_of_labelcount = 0.0
  for t in labelCounts:
    sum_of_labelcount += entropy(t)*sum(t)
  return sum_of_labelcount / data.count()

In [0]:
for k in range(60, 300, 30):
  print(clusteringScore4(data, k))

1.324306956113726
1.1064688883555034
0.8790612095658834
0.3616672324031876
0.28924629592417994
0.10476604691418735
0.2517526550133862
0.22942454180841804


*Nhìn thấy rằng khi chia 210 cluster có  điểm đánh giá thấp nhất, ta sẽ thực hiện phân tích bất thường với cụm này*


---
---

Bắt đầu Anomoly Detection

In [0]:
pipelineModel = fitPipeline4(data, 180)

In [0]:
countByClusterLabel = pipelineModel.transform(data).select("cluster", "label").groupBy("cluster", "label").count().orderBy("cluster", "label")
countByClusterLabel.show()

+-------+----------------+------+
|cluster|           label| count|
+-------+----------------+------+
|      0|         normal.|     9|
|      0|          smurf.|280773|
|      1|        neptune.|   101|
|      1|      portsweep.|     1|
|      2|           imap.|     7|
|      2|        neptune.|   105|
|      3|        neptune.| 36557|
|      3|      portsweep.|    13|
|      4|        neptune.|   101|
|      4|      portsweep.|     4|
|      5|        neptune.|    89|
|      5|          satan.|     1|
|      6|        ipsweep.|     1|
|      6|        neptune.|   102|
|      6|         normal.|     1|
|      6|      portsweep.|     1|
|      7|        neptune.|    25|
|      8|buffer_overflow.|     6|
|      8|      ftp_write.|     4|
|      8|     loadmodule.|     1|
+-------+----------------+------+
only showing top 20 rows



In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.linalg import Vectors

kMeansModel = pipelineModel.stages[-1]
centroids = kMeansModel.clusterCenters()
clustered = pipelineModel.transform(data)

In [0]:
centroids[0]

array([0.00000000e+00, 2.43372987e+00, 0.00000000e+00, 2.07710517e+00,
       0.00000000e+00, 0.00000000e+00, 2.93369918e-04, 2.37881802e+00,
       0.00000000e+00, 0.00000000e+00, 3.93829844e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.40399592e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.46901027e-04, 2.74181059e-05,
       0.00000000e+00, 0.00000000e+00, 2.57606148e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.57203980e-05,
       0.00000000e+00, 2.05843142e+00, 2.02231056e+00, 0.00000000e+00,
       2.01966736e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [0]:
need_order = clustered.select("cluster", "scaledFeatureVector").rdd.map(lambda cluster: (Vectors.squared_distance(centroids[cluster[0]], cluster[1]))).collect()

In [0]:
need_order.sort(reverse=True)
threshold = need_order[100]

In [0]:
originalCols = data.columns

In [0]:
originalCols[0]

'duration'

In [0]:
def distance_cluster(centroids, vec_tor):
  return Vectors.squared_distance(centroids, vec_tor)

In [0]:
anomalies = clustered.rdd.filter(lambda row: distance_cluster(centroids[row["cluster"]], row["scaledFeatureVector"]) > threshold).toDF()

In [0]:
anomalies.head()

Row(duration=1, protocol_type='tcp', service='ftp', flag='SF', src_bytes=60, dst_bytes=189, land=0, wrong_fragment=0, urgent=0, hot=0, num_failed_logins=1, logged_in=0, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=1, srv_count=1, serror_rate=0.0, srv_serror_rate=0.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=1.0, diff_srv_rate=0.0, srv_diff_host_rate=0.0, dst_host_count=151, dst_host_srv_count=47, dst_host_same_srv_rate=0.31, dst_host_diff_srv_rate=0.03, dst_host_same_src_port_rate=0.01, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=0.0, dst_host_srv_serror_rate=0.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0, label='normal.', protocol_type_indexed=1.0, protocol_type_vec=SparseVector(2, {1: 1.0}), service_indexed=8.0, service_vec=SparseVector(65, {8: 1.0}), flag_indexed=0.0, flag_vec=SparseVector(10, {0: 1.0}), featureVector